We will test our intertrack conv1d module here.

It should be initialized to zero, which means at the beginning it should output the result identical to its intra-track counterpart.


# Tensor shape
2d: [B, C, W, H]


3d: [B, T, C, W, H], T = track num

In [1]:
import torch
import torch.nn as nn

batch_size = 16
in_channels = 2
width = 128
height = 128
track_num = 5
device = "cuda"
input_tensor = torch.randn([batch_size, track_num, in_channels, width, height]).to(device)

# Conv1d

Test convolution along the track axis only.

In [2]:
input_1d = input_tensor.permute(3,4,0,2,1).reshape(-1,in_channels,track_num).to(device)
print(input_1d.shape)

torch.Size([262144, 2, 5])


In [3]:
conv1d = nn.Conv1d(in_channels=input_1d.shape[1], out_channels=input_1d.shape[1]*track_num, kernel_size=5).to(device)
conv1d.weight.data.fill_(0.)
conv1d.bias.data.fill_(0.)
params = conv1d.parameters()

In [4]:
output_1d = conv1d(input_1d)
print(output_1d.shape)
output_1d = output_1d.reshape(-1, in_channels,track_num)
res_output_1d = output_1d + input_1d
print(res_output_1d.shape)

torch.Size([262144, 10, 1])
torch.Size([262144, 2, 5])


In [5]:
assert torch.allclose(input_1d, res_output_1d), "Conv1d test failed"